<a href="https://colab.research.google.com/github/sivarohith99/Grant_title_genration/blob/main/t5_small_grant_title_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load the Excel file
file_path = "title_abstract.xlsx"  # Replace with the actual file path
data = pd.read_excel(file_path)

# Display the first few rows to verify the structure
print(data.head())


                                doi  \
0     10.1090/S0002-9939-06-08360-2   
1                 10.1021/JO200213X   
2  10.1111/J.1468-0262.2008.00849.X   
3         10.1017/S0074180900195506   
4          10.1109/TSP.2015.2449255   

                                               title  \
0  A note on uniformization of riemann surfaces b...   
1  With a little help from my friends: forty year...   
2                    Political Economy of Mechanisms   
3  WIYN Open Cluster Study: Tidal Interactions in...   
4  A Saddle Point Algorithm for Networked Online ...   

                                            abstract  
0  We clarify that the Ricci flow can be used to ...  
1  Over the past 40 years, much of the author's r...  
2  We study the optimal Mirrlees taxation problem...  
3  We present an ongoing study on tidal interacti...  
4  An algorithm to learn optimal actions in conve...  


In [2]:
# Assuming 'data' is your dataset with 'title' and 'abstract' columns

# Use only the 'abstract' as input and the corresponding 'title' and 'abstract' as output
data["input"] = data["abstract"]  # input is the abstract text
data["output"] = data["title"] # output is the title

# Keep only the necessary fields for training
processed_data = data[["input", "output"]]

# Display a sample to verify the format
print(processed_data.head())



                                               input  \
0  We clarify that the Ricci flow can be used to ...   
1  Over the past 40 years, much of the author's r...   
2  We study the optimal Mirrlees taxation problem...   
3  We present an ongoing study on tidal interacti...   
4  An algorithm to learn optimal actions in conve...   

                                              output  
0  A note on uniformization of riemann surfaces b...  
1  With a little help from my friends: forty year...  
2                    Political Economy of Mechanisms  
3  WIYN Open Cluster Study: Tidal Interactions in...  
4  A Saddle Point Algorithm for Networked Online ...  


In [3]:
import json

# Save the processed data to a JSONL file
output_file = "fine_tuning_data.jsonl"

with open(output_file, "w") as f:
    for _, row in processed_data.iterrows():
        # Write each row as a JSON object
        json.dump({"input": row["input"], "output": row["output"]}, f)
        f.write("\n")  # Ensure each JSON object is on a new line

print(f"Data saved to {output_file}")


Data saved to fine_tuning_data.jsonl


In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
# Verify if the dataset file is valid
with open("fine_tuning_data.jsonl", "r") as file:
    lines = file.readlines()

# Check the first few lines
for i in range(min(5, len(lines))):
    print(lines[i])


{"input": "We clarify that the Ricci flow can be used to give an independent proof of the uniformization theorem of Riemann surfaces.", "output": "A note on uniformization of riemann surfaces by ricci flow"}

{"input": "Over the past 40 years, much of the author's research, both computational and experimental, has involved collaborations. This Perspective describes some of the author's collaborative research in eight different areas of organic and theoretical chemistry: (1) hydrocarbons containing unsaturatively, 1,3-bridged cyclobutane rings, (2) the use of orbital topology for predicting the ground states of diradicals, (3) violations of Hund's rule, (4) the chemistry of phenylnitrenes, (5) tunneling by carbon in organic reactions, (6) the Cope rearrangement and the effect of substituents on it, (7) pyramidalized alkenes, dehydrocubanes, cubyl cation, and octanitrocubane, and (8) the effects of geminal fluorine substitution at C-2 of 1,3-diradicals. Highlighted in this Perspective ar

In [6]:
import torch
from datasets import load_dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Load your dataset from the JSONL file
dataset_path = '/content/fine_tuning_data.jsonl'

# Load dataset using Hugging Face's Datasets library
dataset = load_dataset("json", data_files={"train": dataset_path})

# Select the model to fine-tune (change as needed, e.g., "facebook/bart-base", "google/pegasus-xsum", etc.)
model_name = 't5-small'  # or "facebook/bart-base", "google/pegasus-xsum", etc.
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Tokenize the dataset
def preprocess_function(examples):
    # Tokenize inputs and outputs (abstract -> title)
    inputs = [ex for ex in examples["input"]]
    targets = [ex for ex in examples["output"]]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Preprocess the dataset
train_dataset = dataset["train"].map(preprocess_function, batched=True)

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",  # output directory for model checkpoints
    evaluation_strategy="epoch",  # evaluate every epoch
    learning_rate=2e-5,  # learning rate for fine-tuning
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,  # batch size per device during evaluation
    weight_decay=0.01,  # L2 regularization
    logging_dir="./logs",  # log directory
    save_steps=500,  # save checkpoint every 500 steps
    save_total_limit=2,  # keep only 2 checkpoints
    num_train_epochs=3,  # number of training epochs
    predict_with_generate=True,  # to use generate method for inference
)

# Initialize the Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,  # using train dataset for evaluation as a placeholder
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

# Optional: Test the model after fine-tuning
input_text = "Example abstract to generate title for."
inputs = tokenizer(input_text, return_tensors="pt").to(device)
generated_ids = model.generate(inputs['input_ids'], max_length=50, num_beams=5, early_stopping=True)
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(f"Generated Title: {generated_text}")


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,2.554600,0.496430
2,0.558800,0.474184
3,0.542700,0.470905


Generated Title: Example abstract to generate title for.


In [8]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=08bf96b350f8594588ac8385f78e1ca219b01493be0ba552ee9d8d5a1d10bc2b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [10]:
import pandas as pd

# Load your dataset
df = pd.read_json('/content/fine_tuning_data.jsonl', lines=True)

# Check the columns and the first few rows of the dataset
print(df.columns)
print(df.head())


Index(['input', 'output'], dtype='object')
                                               input  \
0  We clarify that the Ricci flow can be used to ...   
1  Over the past 40 years, much of the author's r...   
2  We study the optimal Mirrlees taxation problem...   
3  We present an ongoing study on tidal interacti...   
4  An algorithm to learn optimal actions in conve...   

                                              output  
0  A note on uniformization of riemann surfaces b...  
1  With a little help from my friends: forty year...  
2                    Political Economy of Mechanisms  
3  WIYN Open Cluster Study: Tidal Interactions in...  
4  A Saddle Point Algorithm for Networked Online ...  


In [12]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.8 MB/s eta 0:00:00


In [14]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
import evaluate  # Import from the 'evaluate' library instead of 'datasets'
import pandas as pd

# Load fine-tuned model and tokenizer
model_name = "/content/fine_tuned_model"  # Specify the path to your fine-tuned model directory
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Load test data
test_data = pd.read_json('/content/fine_tuning_data.jsonl', lines=True)

# Select a sample of test data (e.g., first 10 rows)
test_samples = test_data.head(10)  # Modify the number of rows as needed

# Prepare input (abstracts) and output (titles) from the test data
input_abstracts = test_samples['input'].tolist()
actual_titles = test_samples['output'].tolist()

# Load ROUGE metric for evaluation
rouge = evaluate.load("rouge")

# Generate titles and evaluate
generated_titles = []
for abstract in input_abstracts:
    inputs = tokenizer(abstract, return_tensors="pt", max_length=512, truncation=True, padding="longest")

    # Generate title using the fine-tuned model
    with torch.no_grad():
        generated_ids = model.generate(inputs["input_ids"], max_length=64, num_beams=4, early_stopping=True)

    generated_title = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    generated_titles.append(generated_title)

# Calculate ROUGE score
results = rouge.compute(predictions=generated_titles, references=actual_titles)

# Print the ROUGE scores
print(f"ROUGE-1: {results['rouge1']}")
print(f"ROUGE-2: {results['rouge2']}")
print(f"ROUGE-L: {results['rougeL']}")

# Optionally, you can also print the generated titles vs actual titles
for idx, (gen, actual) in enumerate(zip(generated_titles, actual_titles)):
    print(f"Sample {idx + 1}:")
    print(f"Input (Abstract): {input_abstracts[idx]}")
    print(f"Generated Title: {gen}")
    print(f"Actual Title: {actual}")
    print("-" * 80)


ROUGE-1: 0.2837652867105882
ROUGE-2: 0.13423125211146988
ROUGE-L: 0.27188284418298436
Sample 1:
Input (Abstract): We clarify that the Ricci flow can be used to give an independent proof of the uniformization theorem of Riemann surfaces.
Generated Title: 
Actual Title: A note on uniformization of riemann surfaces by ricci flow
--------------------------------------------------------------------------------
Sample 2:
Input (Abstract): Over the past 40 years, much of the author's research, both computational and experimental, has involved collaborations. This Perspective describes some of the author's collaborative research in eight different areas of organic and theoretical chemistry: (1) hydrocarbons containing unsaturatively, 1,3-bridged cyclobutane rings, (2) the use of orbital topology for predicting the ground states of diradicals, (3) violations of Hund's rule, (4) the chemistry of phenylnitrenes, (5) tunneling by carbon in organic reactions, (6) the Cope rearrangement and the effe